<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 8
## <center> Vowpal Wabbit в задаче прогнозирования популярности статьи на хабре

В этом задании надо побить бенчмарк в [соревновании](https://www.kaggle.com/c/habr-num-bookmarks) на Kaggle Inclass. Как это делать – ограничений нет (кроме, конечно, ручной разметки), прочитать правила можно [тут](https://www.kaggle.com/c/habr-num-bookmarks/rules). Ниже описаны инструкции, как это сделать с Vowpal Wabbit.

Дедлайн: 31 октября 23:59 UTC +3. Решение надо будет загрузить по [ссылке](https://www.dropbox.com/request/g5WOPrxwvcYwADZCuoY7). В этом соревновании нет задачи победить. Цель – побить бенчмарк и продвинуться в [соревновании](https://mlcourse.arktur.io) по прогнозу популярности статьи на Medium. 

In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error

Посмотрим на одну из строчек в JSON-файле: считаем ее с помощью библиотеки json. Эта строчка соответствует [7-ой статье](https://habrahabr.ru/post/7/) на Хабре.

In [3]:
!head -1 ../../data/train.json > ../../data/train1.json

In [6]:
with open('../../data/train1.json') as inp_json:
    first_json = json.load(inp_json)

In [8]:
first_json.keys()

dict_keys(['post_id', 'url', 'domain', 'content', 'polling', 'link_tags', '_timestamp', 'tags', 'meta_tags', 'published', 'hubs', '_id', 'title', 'flow', 'author', 'flags'])

Видим 16 полей, перечислим некоторые из них:
- _id, url - URL статьи
- published – время публикации статьи
- domain – сайт (например, habrahahbr.ru или geektimes.ru)
- title – название статьи
- content – текст статьи
- hubs - перечисление хабов, к которым относится статья
- tags – теги статьи
- author – автор статьи, его ник и ссылка на профиль

In [10]:
first_json

{'_id': 'https://habrahabr.ru/post/7/',
 '_timestamp': 1493192186.0903192,
 'author': {'name': 'Павел Титов',
  'nickname': '@ptitov',
  'url': 'https://habrahabr.ru/users/ptitov'},
 'content': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.<br>\r\n<br>\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.',
 'domain': 'habrahabr.ru',
 'flags': [],
 'flow': None,
 'hubs': [{'id': 'hub/habr',
   'title': 'Хабрахабр',
   'url': 'https://habrahabr.ru/hub/habr/'}],
 'link_tags': {'alternate': 'https://habrahabr.ru/rss/post/7/',
  'apple-touch-icon-precomposed': '/images/favicons/apple-touch-icon-152x152.png',
  'canonical': 'https://habrahabr.ru/post/7/',
  'icon': '/images/favicons/favicon-16x16.png',
  'image_src': 'https://habrahabr.ru/i/habralogo.jpg',
  'stylesheet': 'https://habracdn.net/habr/styles/1493134745/_build/global_main.css'},
 'meta_tags': {'al:android:app_name': 'Habrahabr',
  'al:an

In [11]:
first_json['_id']

'https://habrahabr.ru/post/7/'

In [12]:
first_json['_timestamp']

1493192186.0903192

In [13]:
first_json['url']

'https://habrahabr.ru/post/7/'

In [14]:
first_json['domain']

'habrahabr.ru'

In [47]:
first_json['published']['$date']

'2006-07-15T01:48:00.000Z'

In [16]:
first_json['title']

'Самопроизвольное разлогинивание'

In [17]:
first_json['content']

'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.<br>\r\n<br>\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.'

In [18]:
first_json['polling']

In [19]:
first_json['post_id']

7

In [20]:
first_json['flags']

[]

In [21]:
first_json['hubs']

[{'id': 'hub/habr',
  'title': 'Хабрахабр',
  'url': 'https://habrahabr.ru/hub/habr/'}]

In [31]:
first_json['flow']

In [23]:
first_json['tags']

['логин', 'login']

In [24]:
first_json['author']

{'name': 'Павел Титов',
 'nickname': '@ptitov',
 'url': 'https://habrahabr.ru/users/ptitov'}

In [25]:
first_json['link_tags']

{'alternate': 'https://habrahabr.ru/rss/post/7/',
 'apple-touch-icon-precomposed': '/images/favicons/apple-touch-icon-152x152.png',
 'canonical': 'https://habrahabr.ru/post/7/',
 'icon': '/images/favicons/favicon-16x16.png',
 'image_src': 'https://habrahabr.ru/i/habralogo.jpg',
 'stylesheet': 'https://habracdn.net/habr/styles/1493134745/_build/global_main.css'}

In [27]:
first_json['meta_tags']

{'al:android:app_name': 'Habrahabr',
 'al:android:package': 'ru.habrahabr',
 'al:android:url': 'habrahabr://post/7',
 'al:windows_phone:app_id': '460a6bd6-8955-470f-935e-9ea1726a6060',
 'al:windows_phone:app_name': 'Habrahabr',
 'al:windows_phone:url': 'habrahabr://post/7',
 'apple-mobile-web-app-title': 'Хабрахабр',
 'application-name': 'Хабрахабр',
 'description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.\r\n\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.',
 'fb:app_id': '444736788986613',
 'keywords': 'логин, login',
 'msapplication-TileColor': '#FFFFFF',
 'msapplication-TileImage': 'mstile-144x144.png',
 'og:description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.  Возможно, это происходит при...',
 'og:image': 'https://habrahabr.ru/i/habralogo.jpg',
 'og:title': 'Самопроизвольное разлогинивание'

Загрузим ответы на обучающей выборке.

In [2]:
train_target = pd.read_csv('../../data/train_target.csv',
                          index_col='url')

In [3]:
train_target.head()

,target
url,
https://habrahabr.ru/post/7/,0.693147
https://geektimes.ru/post/11/,1.098612
https://geektimes.ru/post/112/,0.000000
https://geektimes.ru/post/1127/,0.000000
https://geektimes.ru/post/12664/,0.000000


Сформируйте обучающую выборку для Vowpal Wabbit, выберите признаки title, tags, domain, flow, author, и hubs из JSON-файла.
От самого текста для начала просто возьмем его длину: постройте признак content_len – длина текста в миллионах символов.
Также постройте признаки: час и месяц публикации статьи. Еще, конечно же, возьмите ответы на обучающей выборке из `train_target`. Ниже пример того, как могут выглядеть первые две строки нового файла.

In [89]:
!head -2 ../../data/habr_train.vw

0.6931470000000001 |title �������������������������������� ���������������������������� |tags ���������� login |domain habrahabr.ru |flow None |author @ptitov |hubs ������������������ |num content_len:0.000199 month:7 hour:1
1.0986120000000001 |title Stand-along c������������������ ������������ ������������������ �� ������������ �������������������� ���������� |tags �������������������� ����������������-�������������������� �������������������� �������� �������������� ���������� |domain geektimes.ru |flow None |author @AlexBruce |hubs ������������ �������� |num content_len:0.000988 month:7 hour:14


In [23]:
!head -2 ../../data/habr_train.vw

0.6931470000000001 |title Самопроизвольное разлогинивание |tags логин login |domain habrahabr.ru |flow None |author @ptitov |hubs Хабрахабр |num content_len:0.0 month:7 hour:1
1.0986120000000001 |title Stand-along cообщества против сообществ в рамках социальных сетей |tags сообщества интернет-сообщество социальные сети нишевой бренд |domain geektimes.ru |flow None |author @AlexBruce |hubs Чёрная дыра |num content_len:0.0 month:7 hour:14


In [32]:
from dateutil import parser
import re

def format_vw_obj(obj, label=1):
    tags = " ".join(obj['tags']).replace(':', ' ')
    hubs = " ".join([hub['title'] for hub in obj['hubs']]).replace(':', ' ')
    title = obj['title'].replace(':', ' ')
    pub = parser.parse(obj['published']['$date'])
    morning = 1 if pub.hour < 11 else 0
    evening = 1 if pub.hour > 20 else 0
    return "{} |title {} |tags {} |domain {} |flow {} |author {} |hubs {} |num content_len:{} month:{} hour:{} ev:{} mor:{}" \
            .format(label, title, tags, obj['domain'], obj['flow'], obj['author']['nickname'], hubs, round(len(obj['content'])/10**6, 2), pub.month, pub.hour, evening, morning)

In [33]:
!wc -l '../../data/train.json'

120000 ../../data/train.json


In [34]:
with open('../../data/train.json', encoding="utf8") as inp_json, \
     open('../../data/habr_train.vw', 'w', encoding="utf8") as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        line = format_vw_obj(data_json, train_target.loc[data_json['url']].target)
        out_vw.write(line + '\n')

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [35]:
!split -l 60000 '../../data/habr_train.vw' '../../data/habr_train_'

In [36]:
!mv ../../data/habr_train_aa ../../data/my_train.vw
!mv ../../data/habr_train_ab ../../data/my_valid.vw

In [37]:
!wc -l ../../data/my_train.vw
!wc -l ../../data/my_valid.vw

60000 ../../data/my_train.vw
60000 ../../data/my_valid.vw


In [38]:
!cut -f 1 -d ' ' ../../data/my_valid.vw > ../../data/my_valid_labels.txt

Проделайте все то же с тестовой выборкой, вместо ответов подсовывая что угодно, например, единицы.

In [39]:
with open('../../data/test.json', encoding="utf8") as inp_json, \
     open('../../data/habr_test.vw', 'w', encoding="utf8") as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        line = format_vw_obj(data_json)
        out_vw.write(line + '\n')

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [40]:
!mv ../../data/habr_test.vw ../../data/my_test.vw

Выбор того, как валидировать модель, остается за Вами. Проще всего, конечно, сделать отложенную выборку. Бенчмарк, который Вы видите в соревновании (**vw_baseline.csv**) и который надо побить, получен с Vowpal Wabbit, 3 проходами по выборке (не забываем удалять кэш), биграммами и настроенными гиперпараметрами `bits`, `learning_rate` и `power_t`. 

In [4]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error
import itertools
import os
from time import time

In [5]:
y_valid = np.loadtxt('../../data/my_valid_labels.txt')

In [6]:
def train_vw_model(train_vw_file, model_filename, ngram=1, passes=1, seed=17, bit=18, learning_rate=0.5, power_t=0.5, quiet=False):
    init_time = time()
    vw_call_string = ('vw -d {} -f {} --loss_function quantile --random_seed {seed}') \
                     .format(train_vw_file, model_filename, seed=seed)
    if ngram > 1:
         vw_call_string += ' --ngram {}'.format(ngram)
            
    if passes > 1:
         vw_call_string += ' -k --passes {} --cache_file {}'.format(passes, 
                            model_filename.replace('.vw', '.cache'))
          
    if bit > 0:
        vw_call_string += ' -b {}'.format(bit)
        
    if learning_rate > 0:
        vw_call_string += ' -l {}'.format(learning_rate)
        
    if power_t > 0:
        vw_call_string += ' --power_t {}'.format(power_t)
        
    if quiet:
        vw_call_string += ' --quiet'
    
    
    print('\n', vw_call_string) 
    res = os.system(vw_call_string)
    print('Success. Elapsed: {} sec.'.format(round(time() - init_time, 3))
          if not res else 'Failed.')

In [7]:
def test_vw_model(model_filename, test_vw_file, prediction_filename, true_labels, seed=17, quiet=False):
    init_time = time()
    vw_call_string = ('vw -t -i {} -d {} -p {} --random_seed {}') \
                     .format(model_filename, test_vw_file, prediction_filename, seed)
    if quiet:
        vw_call_string += ' --quiet'
        
#     print(vw_call_string) 
    res = os.system(vw_call_string)
    
    if not res: # the call resulted OK
        vw_pred = np.loadtxt(prediction_filename)
        print("MAE: {}. Elapsed: {} sec.".format(
            round(mean_absolute_error(true_labels, vw_pred), 2), 
            round(time() - init_time, 3)))
    else:
        print('Failed.')

In [8]:
bits = [26]
learning_rate = np.linspace(0.9, 1.1, 6)
power_t = np.linspace(0.7, 1, 6)

for i, (bit, lear_rate, power_t_arg) in enumerate(itertools.product(bits, learning_rate, power_t)):
    train_vw_model('../../data/my_train.vw', '../../data/my_model_train.vw', 
                   ngram=2, passes=3, seed=17, quiet=False, bit=bit, learning_rate=lear_rate, power_t=power_t_arg)
    test_vw_model(model_filename='../../data/my_model_train.vw', test_vw_file='../../data/my_valid.vw', 
              prediction_filename='../../data/my_pred_train.vw',
              true_labels=y_valid, seed=17)


 vw -d ../../data/my_train.vw -f ../../data/my_model_train.vw --loss_function quantile --random_seed 17 --ngram 2 -k --passes 3 --cache_file ../../data/my_model_train.cache -b 26 -l 0.9 --power_t 0.7
Success. Elapsed: 5.954 sec.
MAE: 0.93. Elapsed: 0.894 sec.

 vw -d ../../data/my_train.vw -f ../../data/my_model_train.vw --loss_function quantile --random_seed 17 --ngram 2 -k --passes 3 --cache_file ../../data/my_model_train.cache -b 26 -l 0.9 --power_t 0.76
Success. Elapsed: 5.731 sec.
MAE: 0.94. Elapsed: 0.83 sec.

 vw -d ../../data/my_train.vw -f ../../data/my_model_train.vw --loss_function quantile --random_seed 17 --ngram 2 -k --passes 3 --cache_file ../../data/my_model_train.cache -b 26 -l 0.9 --power_t 0.82
Success. Elapsed: 5.758 sec.
MAE: 0.96. Elapsed: 0.714 sec.

 vw -d ../../data/my_train.vw -f ../../data/my_model_train.vw --loss_function quantile --random_seed 17 --ngram 2 -k --passes 3 --cache_file ../../data/my_model_train.cache -b 26 -l 0.9 --power_t 0.88
Success. Elaps

Success. Elapsed: 5.513 sec.
MAE: 0.93. Elapsed: 0.676 sec.

 vw -d ../../data/my_train.vw -f ../../data/my_model_train.vw --loss_function quantile --random_seed 17 --ngram 2 -k --passes 3 --cache_file ../../data/my_model_train.cache -b 26 -l 1.1 --power_t 0.76
Success. Elapsed: 5.61 sec.
MAE: 0.94. Elapsed: 0.656 sec.

 vw -d ../../data/my_train.vw -f ../../data/my_model_train.vw --loss_function quantile --random_seed 17 --ngram 2 -k --passes 3 --cache_file ../../data/my_model_train.cache -b 26 -l 1.1 --power_t 0.82
Success. Elapsed: 5.596 sec.
MAE: 0.96. Elapsed: 0.655 sec.

 vw -d ../../data/my_train.vw -f ../../data/my_model_train.vw --loss_function quantile --random_seed 17 --ngram 2 -k --passes 3 --cache_file ../../data/my_model_train.cache -b 26 -l 1.1 --power_t 0.88
Success. Elapsed: 5.643 sec.
MAE: 0.98. Elapsed: 0.657 sec.

 vw -d ../../data/my_train.vw -f ../../data/my_model_train.vw --loss_function quantile --random_seed 17 --ngram 2 -k --passes 3 --cache_file ../../data/my

In [12]:
!vw -d ../../data/my_train.vw -f ../../data/my_model_test.vw --loss_function quantile --random_seed 17 --ngram 2 -k --passes 3 --cache_file ../../data/my_model_train.cache -b 26 -l 1 --power_t 0.7
test_vw_model(model_filename='../../data/my_model_test.vw', test_vw_file='../../data/my_valid.vw', 
              prediction_filename='../../data/my_pred_test.vw',
              true_labels=y_valid, seed=17)

Generating 2-grams for all namespaces.
final_regressor = ../../data/my_model_test.vw
Num weight bits = 26
learning rate = 1
initial_t = 0
power_t = 0.7
decay_learning_rate = 1
creating cache_file = ../../data/my_model_train.cache
Reading datafile = ../../data/my_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.346574 0.346574            1            1.0   0.6931   0.0000       17
0.264460 0.182346            2            2.0   1.0986   0.7339       37
0.200145 0.135831            4            4.0   0.0000   0.2231       45
0.672408 1.144670            8            8.0   4.9698   0.3764       41
0.822268 0.972128           16           16.0   2.3026   1.2408       33
0.961349 1.100430           32           32.0   2.8904   3.6216       25
0.847553 0.733756           64           64.0   0.0000   2.1487       21
0.738753 0.629953          128          128.0   5.1475 

In [25]:
!vw -d ../../data/habr_train.vw -f ../../data/my_model.vw --loss_function quantile --random_seed 17 --ngram 2 -k --passes 3 --cache_file ../../data/my_model_train.cache -b 26 -l 1 --power_t 0.85 --quiet
!vw -i ../../data/my_model.vw -t -d ../../data/my_test.vw -p ../../data/habr_pred.vw --quiet

In [26]:
sample_sub = pd.read_csv('../../data/sample_submission.csv', 
                         index_col='url')

In [27]:
with open('../../data/habr_pred.vw', encoding="utf8") as f:
    preds = np.array([float(val) for val in f.readlines()])

In [28]:
your_submission = sample_sub.copy()
your_submission['target'] = preds
your_submission.to_csv('../../data/submission.csv')

Для получения баллов в #mlcourse_open команда (из 1 человека) должна называться в точном соответствии с тем, как оно записано в рейтинге.